# Koppelen van borden- en wegenbestanden
Wegenbestand wordt omgezet naar een directed graph met de relevante restrictie-type verkeersborden (type-C) gekoppeld aan elk wegsegment. Deze graph is geschikt voor een kortste pad algoritme. Deze dataset is het basis wegenbestand die gebruikt wordt voor diverse casussen binnen dit project.

In [1]:
import psycopg2
try:
    # get secret values from keyvault
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    # make connection
    conn = psycopg2.connect(dbname="rdt",user="user",host="host",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

In [2]:
subselection_string="('Noord-Holland','Flevoland')"
# subselection_string="('Noord-Holland','Flevoland','Utrecht')"

Selectie van gemeentes in het interessegebied. Sommige koppelingen schalen zeer nadelig, door opslitsing per gemeente kan de rekentijd acceptabel gehouden worden

In [3]:
sql=f'''
SELECT  g.statcode
FROM    stg_ndw_wegen.cbs_gemeentegrenzen_2023 g,
        stg_ndw_wegen.cbs_provinciegrenzen_2023 p
WHERE   ST_Within(g.geom,p.geom)
AND     p.statnaam IN {subselection_string}'''
cur.execute(sql)
data=cur.fetchall()
data = [item for sublist in data for item in sublist]
# print(data)

## Make subselection

In [3]:
sql=f'''drop table if exists stg_ndw_wegen.nwb_raw_subselection;
CREATE TABLE stg_ndw_wegen.nwb_raw_subselection
AS (
    select w.*
    FROM (
            SELECT *, ST_GeomFromText(shape,28992) as geom
            FROM stg_ndw_wegen.nwb_raw
        ) as w
        ,(
            SELECT ST_Union(ARRAY_AGG(geom)) as geom
            FROM stg_ndw_wegen.cbs_provinciegrenzen_2023
            WHERE statnaam IN {subselection_string}
            GROUP BY rubriek
        ) as p
    WHERE ST_Within(w.geom,p.geom)
);'''
cur.execute(sql)

In [4]:
sql=f'''drop table if exists stg_ndw_borden.ndw_bebording_raw_geom_subselection;
CREATE TABLE stg_ndw_borden.ndw_bebording_raw_geom_subselection
AS (
    SELECT bc.*
    FROM (
            SELECT *, ST_transform(ST_SetSRID(ST_MakePoint("location.wgs84.longitude"::float, "location.wgs84.latitude"::float), 4326),28992) as geom
            FROM stg_ndw_borden.ndw_bebording_raw_catc
            WHERE black_code NOT IN ('inapplicable','Inapplicable')
            AND black_code != '21-7.5' -- verkeerd uitgelezen zone-eind bord
        ) as bc
        ,(
            SELECT ST_Union(ARRAY_AGG(geom)) as geom
            FROM stg_ndw_wegen.cbs_provinciegrenzen_2023
            WHERE statnaam IN {subselection_string}
            GROUP BY rubriek
        ) as p
    WHERE ST_Within(bc.geom,p.geom)
    );'''
cur.execute(sql)

## Wegenbestand

In [5]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_directed;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_directed
AS (
    select w.wvk_id*-1 as id,
        w.jte_id_beg  as source,
        w.jte_id_end as target,
        frc as frc,
        1 as speed,  
        geom as geom
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng in ('B','H','O') 
    union all 
    select w.wvk_id as id,
        w.jte_id_end   as source, -- dit is bewust omgedraaid 
        w.jte_id_beg   as target, -- dit is bewust omgedraaid
        frc as frc, 
        1 as speed, 
        st_reverse(geom) as geom -- dit is bewust omgedraaid
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng in ('B','O')
    );'''
cur.execute(sql)
sql='''CREATE INDEX nwb_wegvakken_directed_geomidx
	ON prd_ndw_wegen.nwb_wegvakken_directed
  	USING GIST (geom);
    ANALYZE prd_ndw_wegen.nwb_wegvakken_directed;'''
cur.execute(sql)

In [6]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_nodes;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_nodes
AS (
    select source as node,
        st_startpoint(st_linemerge(geom)) as geom
    from prd_ndw_wegen.nwb_wegvakken_directed
    union all 
    select  target as node,
        st_endpoint(st_linemerge(geom)) as geom
    from prd_ndw_wegen.nwb_wegvakken_directed
    );'''
cur.execute(sql)

In [7]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_oneway;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_oneway
AS (
    select w.wvk_id as id,
        w.jte_id_beg  as source,
        w.jte_id_end as target,
        frc as frc, 
        geom as geom
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng ='H'
);'''
cur.execute(sql)

## Bordenbestand

In [8]:
sql='''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_1;
CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_1
AS (
	select b.id as bord_id,
		-- case when b.textsigns is null then 'verbod' else 'onbekend'  end as geldigheid,
		case when text_signs = '{}' then 'verbod' when text_signs = '{""}' then 'verbod' else 'onbekend'  end as geldigheid,
		b.rvv_code,
		b.text_signs as bord_text,
		b.black_code as tekst_waarde,
		b."location.wgs84.latitude" as lat,
		b."location.wgs84.longitude" as lon, 
		case when b."location.side" = 'N'  then 0
			when b."location.side" = 'O'  then 90
			when b."location.side" = 'Z'  then 180
			when b."location.side" = 'W'  then 270
			else 0 end as  kijkrichting,
		b."location.road.name" as straatnaam,
		b."details.image" as url_foto,
		concat ('https://www.google.com/maps/place/',b."location.wgs84.latitude",'+',b."location.wgs84.longitude") as google_url,
		b."location.road.wvk_id" as nwb_wegvak_id_volgens_ndw,
		geom as geom
	from stg_ndw_borden.ndw_bebording_raw_geom_subselection AS b
	order by b.text_signs desc
	);'''
cur.execute(sql)
sql='''CREATE INDEX ndw_bebording_verrijkt_1_geomidx
	ON int_ndw_borden.ndw_bebording_verrijkt_1
  	USING GIST (geom);
	ANALYZE int_ndw_borden.ndw_bebording_verrijkt_1;'''
cur.execute(sql)

In [12]:
sql='''drop table if exists int_ndw_wegen.nwb_wegvakken_directed_offset;
CREATE TABLE int_ndw_wegen.nwb_wegvakken_directed_offset
AS (
    SELECT w.*, ST_OffsetCurve(geom,-0.1) as geom_offset
    FROM prd_ndw_wegen.nwb_wegvakken_directed w
);'''
cur.execute(sql)

In [7]:
for gm_code in data:
    print(gm_code)
    sql_sub=f'''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_1_sub;
    CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_1_sub
    AS (
        SELECT  b.*
        FROM    int_ndw_borden.ndw_bebording_verrijkt_1 b,
                stg_ndw_wegen.cbs_gemeentegrenzen_2023 g
        WHERE   ST_Within(b.geom,g.geom)
        AND     g.statcode = '{gm_code}'
    );'''
    cur.execute(sql_sub)
    sql_sub2=f'''drop table if exists int_ndw_wegen.nwb_wegvakken_directed_offset_sub;
    CREATE TABLE int_ndw_wegen.nwb_wegvakken_directed_offset_sub
    AS (
        SELECT  w.*
        FROM    int_ndw_wegen.nwb_wegvakken_directed_offset w,
                stg_ndw_wegen.cbs_gemeentegrenzen_2023 g
        WHERE   ST_Within(w.geom,g.geom)
        AND     g.statcode = '{gm_code}'
    );'''
    cur.execute(sql_sub2)
    sql=f'''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code};
    CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code}
    AS (
        select 
            bord_id, geldigheid, lat, lon, kijkrichting, straatnaam, url_foto, google_url, nwb_wegvak_id_volgens_ndw, geom,
            rvv_code as rvv_modelnummer, 
            bord_text as onderbord_tekst,
            tekst_waarde, 
            id as _wegvak_id_volgens_script,
            id as link_gevalideerd
        from (
            SELECT b.*
                ,roads_closest.ID::INT
                ,roads_closest.afstand
            FROM int_ndw_borden.ndw_bebording_verrijkt_1_sub AS b
            CROSS JOIN LATERAL (
                SELECT b.geom <-> n.geom_offset as afstand
                    , n.ID
                FROM int_ndw_wegen.nwb_wegvakken_directed_offset_sub as n
                WHERE st_dwithin(b.geom,n.geom_offset,50)=true
                ORDER BY afstand 
                LIMIT 1
            ) as roads_closest
        ) as sub
    );'''
    cur.execute(sql)

In [10]:
sql_union='''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_2;
CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_2
AS ('''
gm_code_last = data[-1]
for gm_code in data:
    sql_union+=f'''select *
    from int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code}
    '''
    if not gm_code == gm_code_last:
        sql_union+='''union all
        '''
sql_union+=''');'''
# print(sql_union)
cur.execute(sql_union)

In [11]:
sql='''drop table if exists prd_ndw_borden.ndw_bebording_verrijkt_3;
CREATE TABLE prd_ndw_borden.ndw_bebording_verrijkt_3
AS (
    select bord_id,geldigheid,lat,lon,kijkrichting,straatnaam,url_foto,google_url,nwb_wegvak_id_volgens_ndw,geom,rvv_modelnummer,onderbord_tekst,_wegvak_id_volgens_script,link_gevalideerd,tekst_waarde
    FROM int_ndw_borden.ndw_bebording_verrijkt_2 AS b2
);'''
cur.execute(sql)
# No longer needed, as base dataset now containt tekst_waarde
# Kepts as a copy of bebording_verrijkt_2, for legacy purposes

In [6]:
sql='''-- verrijkte borden koppelen aan netwerk 
drop table if exists prd_ndw_borden.nwb_wegvakken_bebording;
CREATE TABLE prd_ndw_borden.nwb_wegvakken_bebording
AS (
	select
		sub.id::int,
		sub.source::int,
		sub.target::int,
		sub.speed as spd, 
		case
			when speed is null then -1
			when speed = 0 then -1
			else st_length(geom)/(speed/3.6) 
		end as cost,
		c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17, c18, c19, c20, c21,
		sub.geom
	from (
		select  id , source, target,  speed,  st_transform(geom,4326) as geom
		from prd_ndw_wegen.nwb_wegvakken_directed
			) as sub
		left join (
			select  vb.link_gevalideerd as linknr_vma,
				min(case when vb.rvv_modelnummer = 'C1' then 1 else NULL end) as c01,
				min(case when vb.rvv_modelnummer = 'C6' then 1 else NULL end) as c06,
				min(case when vb.rvv_modelnummer LIKE 'C7%' AND vb.rvv_modelnummer NOT LIKE 'C7A%' AND vb.rvv_modelnummer NOT LIKE 'C7B%' then 1 else NULL end) as c07,
				min(case when vb.rvv_modelnummer = 'C7A' then 1 else NULL end) as c07a,
				min(case when vb.rvv_modelnummer = 'C7B' then 1 else NULL end) as c07b,
				min(case when vb.rvv_modelnummer = 'C8' then 1 else NULL end) as c08,
				min(case when vb.rvv_modelnummer = 'C9' then 1 else NULL end) as c09,
				min(case when vb.rvv_modelnummer = 'C10' then 1 else NULL end) as c10,
				min(case when vb.rvv_modelnummer = 'C11' then 1 else NULL end) as c11,
				min(case when vb.rvv_modelnummer = 'C12' then 1 else NULL end) as c12,
				min(case when vb.rvv_modelnummer = 'C17' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c17,
				min(case when vb.rvv_modelnummer = 'C18' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c18,
				min(case when vb.rvv_modelnummer = 'C19' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c19,
				min(case when vb.rvv_modelnummer = 'C20' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c20,
				min(case when vb.rvv_modelnummer = 'C21' or vb.rvv_modelnummer = 'C21_ZB' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c21
			from  prd_ndw_borden.ndw_bebording_verrijkt_3 AS vb
			where  vb.link_gevalideerd  <> 0
			and vb.tekst_waarde NOT LIKE 'Z%'
			and vb.tekst_waarde NOT IN ('Inapplicable','inapplicable')
			and vb.tekst_waarde != '21-7.5' -- verkeerd uitgelezen zone-eind bord
			and geldigheid in ('verbod')
			group by  vb.link_gevalideerd
			) as borden
		on sub.id=borden.linknr_vma
	);'''
cur.execute(sql)
sql='''CREATE INDEX nwb_wegvakken_bebording_geomidx
	ON prd_ndw_borden.nwb_wegvakken_bebording
  	USING GIST (geom);
	ANALYZE prd_ndw_borden.nwb_wegvakken_bebording;'''
cur.execute(sql)